In [1]:
#-----------------------------------------
# importamos los paquetes que vamos a usar
#-----------------------------------------

import pypulse as pulsar

import numpy as np
import glob                         # para hacer listas de archivo
import subprocess                   # para usar subprocesos

import glob
import subprocess
import os
import shutil

from IPython.display import Image

In [2]:
ant = 'A1'                                              # antena usada (A1 o A2)

In [ ]:
#----------------------------------------------------------------------------
# Cargamos las observaciones a Pypulse
#----------------------------------------------------------------------------           

# creamos una lista con todos los archivos .pfd de observaciones

os.chdir('./' + ant + '/')                              # entramos al directorio que contiene los .pfds
pfd_files = glob.glob('*pfd')                           # lista de archivos .pfd

# Convert PFD files to PSRFITS

for pfd in pfd_files:
    subprocess.check_call(['psrconv','-o','PSRFITS','-e','fits',pfd])   # convertimos el archivo a .fits
    
# Save all PSRFITS files
psrfits_files = glob.glob('*fits')

# Load all PSRFITS into PyPulse
fits_pypulse= []
for psrfits in psrfits_files:
    temp_pypulse= pulsar.Archive(psrfits)
    fits_pypulse.append(temp_pypulse)

observations = dict(zip(psrfits_files, fits_pypulse))

# Create single pulse object
# Creat array of best profiles

single_pulses=[]

for observation in fits_pypulse:
    
    # First crunch in time and frequency
    observation.fscrunch()
    observation.tscrunch()
    
    # Now get array of the best profile
    tmp_singlepulse = observation.getSinglePulses(windowsize=256)
    
    # Align and normalize
    tmp_singlepulse.center_align()
    tmp_singlepulse.normalize()
    
    # Save aligned and normalize in each observation
    single_pulses.append(tmp_singlepulse)
    
# Save single pulse object
sp_observations = dict(zip(psrfits_files, single_pulses))

os.chdir('..')                                 # salimos del directorio que contiene los .pfds

In [ ]:
#----------------------------------------------------------------------------
# Ahora buscamos la osbervación de mayor S/N y la usamos como template
#----------------------------------------------------------------------------

sn = np.zeros(len(observations))               # creamos un array que contenga el S/N de todas las observaciones

i=0
for key in observations:                       # barremos todas las observaciones
    sn[i] = sp_observations[key].getSN()       # calculamos su S/N
    i += 1

maxsn = sn[np.argmax((sn))]                    # buscamos el mayor valor de S/N
obs_maxsn = psrfits_files[np.argmax((sn))]     # y el nombre de la respectiva observación
    
print( "La observación de mayor S/N es: " + str( maxsn ) )   # Mostramos los resultados
print( "Su S/N es: " + str( obs_maxsn ) )

if not os.path.exists('./templates'):          # creamos una carpeta para guardar el template
    os.mkdir('./templates')

shutil.copy('./' + ant + '/' + str(obs_maxsn) , './templates/' +)  # copiamos el archivo a la carpeta de los templates

subprocess.call( "psrsmooth -W " + str(obs_maxsn), cwd='./templates/', shell=True ) # suavizamos la observación

os.rename('./templates/' + str(obs_maxsn) + '.sm' , "./templates/template_" + ant + ".std" ) # usamos esta convención para el nombre

In [5]:
# Escribimos los nombres de las observaciones usadas como templates en un archivo de salida,
# de modo que los otros programas sepan qué observaciones excluir de cálculo de TOAs

file = obs_maxsn.replace(".fits", ".pfd")                        # arreglamos el nombre del archivo para que en lugar de .fits diga .pfd

f = open('./templates/obs_template_' + ant + '.dat', "w+")       # abrimos el nuevo archivo .tim
print(file, file = f)
f.close()